In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights

# weights 파라미터를 사용하여 사전 학습된 모델 로드 (권장 방식)
weights = ResNet50_Weights.IMAGENET1K_V1  # 또는 ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)


# -------------------------------
# 1. 커스텀 데이터셋 정의
# -------------------------------
class KFoodDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): kfood 최상위 폴더 경로 (하위에 여러 단계 폴더가 있음)
            transform (callable, optional): 이미지에 적용할 변환 함수
        """
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.classes = set()

        # 최상위 폴더부터 재귀적으로 모든 이미지 파일 탐색
        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    # 마지막 폴더 이름을 라벨로 사용합니다.
                    label = os.path.basename(root)
                    path = os.path.join(root, file)
                    self.image_paths.append(path)
                    self.labels.append(label)
                    self.classes.add(label)

        # 클래스 목록 정렬 후, 클래스 이름 -> 인덱스 매핑 생성
        self.classes = sorted(list(self.classes))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        # 문자열 라벨을 인덱스로 변환
        self.labels = [self.class_to_idx[label] for label in self.labels]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img = Image.open(self.image_paths[index]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[index]
        return img, label


# -------------------------------
# 2. 하이퍼파라미터 및 설정
# -------------------------------
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.001

# kfood 최상위 폴더 경로만 지정
kfood_dir = "C:/Users/kimyo/Downloads/kfood"  # 실제 경로로 수정하세요

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------------------
# 3. 데이터 전처리(transform) 정의
# -------------------------------
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# -------------------------------
# 4. 데이터셋 및 DataLoader 구성
# -------------------------------
dataset = KFoodDataset(root_dir=kfood_dir, transform=transform_train)
# 전체 데이터를 8:2 비율로 분리 (예시)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Windows 환경에서는 num_workers=0 또는 main guard를 사용해야 합니다.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

NUM_CLASSES = len(dataset.classes)
print(f"Number of classes: {NUM_CLASSES}")
print(f"Total dataset size: {len(dataset)} images")
print(f"Train dataset size: {train_size} images")
print(f"Validation dataset size: {val_size} images")

# -------------------------------
# 5. ResNet 모델 구성 (마지막 fc layer 재정의)
# -------------------------------
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)  # 마지막 출력층 재정의
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


# -------------------------------
# 6. 학습 및 검증 함수 정의
# -------------------------------
def train_epoch(model, dataloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    print(f"Epoch {epoch + 1} - Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


def validate_epoch(model, dataloader, criterion, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    print(f"Epoch {epoch + 1} - Validation Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    return epoch_loss


# -------------------------------
# 7. 모델 학습 실행
# -------------------------------
if __name__ == '__main__':
    best_loss = float("inf")
    for epoch in range(EPOCHS):
        train_epoch(model, train_loader, criterion, optimizer, epoch)
        val_loss = validate_epoch(model, val_loader, criterion, epoch)

        # 검증 손실이 개선되면 모델 저장
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), "best_resnet50_kfood.pth")
            print(f"Saved best model at epoch {epoch + 1}")

    print("Training completed!")


In [ ]:
import os
import requests
import json
import pandas as pd

# Azure Custom Vision API 정보
PREDICTION_KEY = "6XRfesg0Ka3EwuWvdx1MuMY9gGij25QJP3eoMKC7OGPQOtl9EojMJQQJ99BBACi0881XJ3w3AAAIACOGENeY"
PROJECT_ID = "bacf31b3-a6f7-40fd-8357-23aace5429cd"
ITERATION_NAME = "Iteration1"
ENDPOINT = "https://foodnamepredict-prediction.cognitiveservices.azure.com"

# API 엔드포인트 URL 설정
PREDICTION_URL = f"{ENDPOINT}/customvision/v3.0/Prediction/{PROJECT_ID}/classify/iterations/{ITERATION_NAME}/image"

# 테스트할 이미지 절대 경로로 변경 (Windows)
image_path = "Img_010_0021.jpg"

# 파일 존재 확인
if not os.path.exists(image_path):
    print(f"파일을 찾을 수 없습니다: {image_path}")
    exit()  # 프로그램 종료

# 요청 헤더 설정
headers = {
    "Prediction-Key": PREDICTION_KEY,
    "Content-Type": "application/octet-stream"
}

# API 호출
with open(image_path, "rb") as image_file:
    response = requests.post(PREDICTION_URL, headers=headers, data=image_file)

# API 응답을 JSON으로 변환
result = response.json()

# JSON 응답 출력 (Pretty Print)
print("JSON:")
print(json.dumps(result, indent=4))

# 예측 결과가 있는지 확인 후, 가장 높은 확률을 가진 태그 선택
if "predictions" in result and len(result["predictions"]) > 0:
    best_prediction = max(result["predictions"], key=lambda x: x["probability"])

    # 바로 사용 가능한 값으로 저장
    predicted_tag = best_prediction["tagName"]
    confidence_score = best_prediction["probability"] * 100  # 0.92 → 92.0%

    # CSV 파일 로드 (칼로리 정보를 가져오기 위해)
    CSV_FILE_PATH = "250218 한국음식 데이터 합본.csv"

    try:
        df = pd.read_csv(CSV_FILE_PATH)
    except Exception as e:
        print(f"CSV 파일 로드 오류: {e}")
        exit()

    # 예측된 음식명과 일치하는 칼로리 값 찾기
    matched_row = df[df['통합_식품명'] == predicted_tag]

    if not matched_row.empty:
        calories = matched_row.iloc[0]['1인당Cal']  # 첫 번째 일치 행의 칼로리 값
    else:
        calories = "데이터 없음"

    # 결과 출력
    print(f"\n예측 결과:")
    print(f"음식 이름: {predicted_tag}")
    print(f"신뢰도: {confidence_score:.2f}%")
    print(f"칼로리: {calories}")
else:
    print("\n예측 결과를 찾을 수 없습니다.")


In [3]:
import os
import requests
import json
import pandas as pd
import torch

# Azure Custom Vision API 정보
PREDICTION_KEY = "6XRfesg0Ka3EwuWvdx1MuMY9gGij25QJP3eoMKC7OGPQOtl9EojMJQQJ99BBACi0881XJ3w3AAAIACOGENeY"
PROJECT_ID = "bacf31b3-a6f7-40fd-8357-23aace5429cd"
ITERATION_NAME = "Iteration1"
ENDPOINT = "https://foodnamepredict-prediction.cognitiveservices.azure.com"

# API 엔드포인트 URL 설정
PREDICTION_URL = f"{ENDPOINT}/customvision/v3.0/Prediction/{PROJECT_ID}/classify/iterations/{ITERATION_NAME}/image"

# 테스트할 이미지 절대 경로로 변경 (Windows)
image_path = "Img_010_0021.jpg"

# 파일 존재 확인
if not os.path.exists(image_path):
    print(f"파일을 찾을 수 없습니다: {image_path}")
    exit()  # 프로그램 종료

# 요청 헤더 설정
headers = {
    "Prediction-Key": PREDICTION_KEY,
    "Content-Type": "application/octet-stream"
}

# API 호출
with open(image_path, "rb") as image_file:
    response = requests.post(PREDICTION_URL, headers=headers, data=image_file)

# API 응답을 JSON으로 변환
result = response.json()

# JSON 응답 출력 (Pretty Print)
print("JSON:")
print(json.dumps(result, indent=4))

# 예측 결과가 있는지 확인 후, 가장 높은 확률을 가진 태그 선택
if "predictions" in result and len(result["predictions"]) > 0:
    best_prediction = max(result["predictions"], key=lambda x: x["probability"])

    # 바로 사용 가능한 값으로 저장
    predicted_tag = best_prediction["tagName"]
    confidence_score = best_prediction["probability"] * 100  # 0.92 → 92.0%

    # CSV 파일 로드 (칼로리 정보를 가져오기 위해)
    CSV_FILE_PATH = "250218 한국음식 데이터 합본.csv"

    try:
        food_data = pd.read_csv(CSV_FILE_PATH)
    except Exception as e:
        print(f"CSV 파일 로드 오류: {e}")
        exit()

    # 예측된 음식명과 일치하는 칼로리 값 찾기
    matched_row = food_data[food_data['통합_식품명'] == predicted_tag]

    if not matched_row.empty:
        input_calories = matched_row.iloc[0]['1인당Cal']  # 첫 번째 일치 행의 칼로리 값
    else:
        input_calories = "데이터 없음"

    # 운동 데이터 CSV 로드
    exercise_data = pd.read_csv("운동데이터_송부.csv")

    # 필요한 칼럼만 선택
    exercise_data = exercise_data[['운동이름', '칼로리(1분)', '종류']]

    # 음식이 데이터에 존재하지 않거나 칼로리 값이 "데이터 없음"인 경우
    if input_calories == "데이터 없음":
        print("❌ 해당 음식의 칼로리 데이터를 찾을 수 없습니다.")
    else:
        # 운동 추천: 섭취한 칼로리를 소모할 수 있는 운동 리스트 계산
        exercise_data['운동시간(분)'] = input_calories / exercise_data['칼로리(1분)']

        # 운동 추천 리스트 출력
        print(f"✅ '{predicted_tag}' (섭취 칼로리: {input_calories} kcal)을 소모하려면:")
        print(exercise_data[['운동이름', '운동시간(분)']].sort_values(by='운동시간(분)').head(5))  # 상위 5개 운동 추천
else:
    print("\n예측 결과를 찾을 수 없습니다.")


JSON:
{
    "id": "71586106-33c1-4d85-a10c-62e946687f1c",
    "project": "bacf31b3-a6f7-40fd-8357-23aace5429cd",
    "iteration": "ecfea1fb-a60e-447a-bdff-9be4621b91b1",
    "created": "2025-02-20T01:42:33.309Z",
    "predictions": [
        {
            "probability": 0.85196537,
            "tagId": "81104f2d-b0a7-480d-bbd6-ead4acc0ba76",
            "tagName": "\uc870\uac1c\uad6c\uc774"
        },
        {
            "probability": 0.03253507,
            "tagId": "55b3fb91-4964-4f69-8f02-a00cbdbd191d",
            "tagName": "\uac04\uc7a5\uac8c\uc7a5"
        },
        {
            "probability": 0.024168639,
            "tagId": "5bf9156b-8c25-484c-8d51-eeab92ce3693",
            "tagName": "\uaf2c\ub9c9\ucc1c"
        },
        {
            "probability": 0.01295535,
            "tagId": "87bc367f-d7d5-4ac0-87cc-9f4d0c3e4d16",
            "tagName": "\uc0b0\ub099\uc9c0"
        },
        {
            "probability": 0.008482886,
            "tagId": "8451629a-39f9-4df0-98